In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Load the dataset
data = pd.read_csv('weather.csv')

# Handle missing values
data.dropna(inplace=True)  # Drop rows with missing values

# Select relevant features and target variable
# Example: assuming 'temperature' is the target variable and other columns are features
features = data[['Data.Temperature.Avg Temp', 'Data.Temperature.Max Temp', 'Data.Temperature.Min Temp']]  # Select features
target = data['Data.Precipitation']  # Select target variable

# Scale numerical features
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# Define sequence length (time steps) for LSTM model
seq_length = 10  # Example: use 10 time steps

# Reshape data into sequences
X = []
y = []
for i in range(len(scaled_features) - seq_length):
    X.append(scaled_features[i:i+seq_length])
    y.append(target[i+seq_length])

X = np.array(X)
y = np.array(y)

# Split the dataset into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Print shapes of training and test sets
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (13386, 10, 3)
y_train shape: (13386,)
X_test shape: (3347, 10, 3)
y_test shape: (3347,)


In [18]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

# Define the LSTM-based architecture
model = Sequential()

# Add the first LSTM layer with dropout
model.add(LSTM(units=64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

# Add the second LSTM layer with dropout
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))

# Add the third LSTM layer with dropout
model.add(LSTM(units=64))
model.add(Dropout(0.2))

# Add the output layer
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Display the model summary
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            17408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 10, 64)            33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [19]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

# Define baseline model with a single LSTM layer
baseline_model = Sequential()
baseline_model.add(LSTM(units=64, input_shape=(X_train.shape[1], X_train.shape[2])))
baseline_model.add(Dropout(0.2))
baseline_model.add(Dense(units=1))
baseline_model.compile(optimizer='adam', loss='mean_squared_error')

# Train and evaluate the baseline model
baseline_history = baseline_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Experiment with stacking multiple LSTM layers and adjusting the number of units
models = []

# Model 1: Two LSTM layers with 64 units each
model1 = Sequential()
model1.add(LSTM(units=64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model1.add(LSTM(units=64))
model1.add(Dropout(0.2))
model1.add(Dense(units=1))
model1.compile(optimizer='adam', loss='mean_squared_error')
models.append(model1)

# Model 2: Three LSTM layers with 128 units each
model2 = Sequential()
model2.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model2.add(LSTM(units=128, return_sequences=True))
model2.add(LSTM(units=128))
model2.add(Dropout(0.2))
model2.add(Dense(units=1))
model2.compile(optimizer='adam', loss='mean_squared_error')
models.append(model2)

# Train and evaluate each model
histories = []
for model in models:
    history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
    histories.append(history)


Train on 13386 samples, validate on 3347 samples
Epoch 1/10
13386/13386 [==============================] - 18s 1ms/step - loss: 1.0484 - val_loss: 0.6993
Epoch 2/10
13386/13386 [==============================] - 12s 933us/step - loss: 1.0397 - val_loss: 0.6938
Epoch 3/10
13386/13386 [==============================] - 16s 1ms/step - loss: 1.0384 - val_loss: 0.6898
Epoch 4/10
13386/13386 [==============================] - 13s 943us/step - loss: 1.0351 - val_loss: 0.6870
Epoch 5/10
13386/13386 [==============================] - 11s 815us/step - loss: 1.0339 - val_loss: 0.7064
Epoch 6/10
13386/13386 [==============================] - 11s 794us/step - loss: 1.0348 - val_loss: 0.6850
Epoch 7/10
13386/13386 [==============================] - 10s 750us/step - loss: 1.0309 - val_loss: 0.6844
Epoch 8/10
13386/13386 [==============================] - 9s 665us/step - loss: 1.0299 - val_loss: 0.6851
Epoch 9/10
13386/13386 [==============================] - 9s 663us/step - loss: 1.0305 - val_loss: 0

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Predict the target variable on the test set
y_pred = model.predict(X_test)

# Calculate mean absolute error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

# Calculate root mean squared error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE):", rmse)


In [ ]:
import matplotlib.pyplot as plt

# Plot the actual vs. predicted values
plt.figure(figsize=(10, 6))
plt.plot(y_test, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.title('Actual vs. Predicted Values')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()


In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

# Define a function to create the LSTM model
def create_model(learning_rate=0.001, num_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(units=num_units, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Create KerasRegressor wrapper for use with scikit-learn
model = KerasRegressor(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# Define hyperparameter grid
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'num_units': [32, 64, 128],
    'dropout_rate': [0.2, 0.3, 0.4]
}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_result.best_params_)

# Evaluate the best model on the test set
best_model = grid_result.best_estimator_
loss = best_model.score(X_test, y_test)
print("Test Loss (MSE) with Best Model:", loss)


In [ ]:
#Hyperparameter tuning using Grid Search
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV

# Define a function to create the LSTM model
def create_model(learning_rate=0.001, num_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(units=num_units, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Create KerasRegressor wrapper for use with scikit-learn
model = KerasRegressor(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

# Define hyperparameter grid
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'num_units': [32, 64, 128],
    'dropout_rate': [0.2, 0.3, 0.4]
}

# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_result.best_params_)

# Evaluate the best model on the test set
best_model = grid_result.best_estimator_
loss = best_model.score(X_test, y_test)
print("Test Loss (MSE) with Best Model:", loss)


In [ ]:
1.Discuss the challenges encountered during model training and
optimization.
Ans: Selecting the optimal set of hyperparameters (e.g., learning rate, dropout rate, number of units) for an LSTM model can be challenging.